In [1]:
import networkx as nx
import numpy as np
import random
import osmnx as ox
import folium
from folium.plugins import MarkerCluster
import webbrowser
import requests
from itertools import permutations
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point
import os.path
import json
import matplotlib.pyplot as plt
from bmi_topography  import Topography
from geopy.geocoders import Nominatim
from IPython.display import display

In [2]:
API_KEY = 'x'
API_KEY_elevation = "x"

In [3]:
# Fonction pour obtenir les adresses pour les points
def get_addresses_for_points(points):
    addresses = []
    for point in points:
        lat, lon = point
        url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=18&addressdetails=1'
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            display_name = data.get('display_name', 'Adresse non trouvée')
            addresses.append(display_name)
        except requests.exceptions.RequestException as e:
            print(f"Une erreur s'est produite lors de la requête : {e}")
            addresses.append(None)
    return addresses

# Fonction pour obtenir les élévations pour les points
def get_city_topography(city_name, points):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(city_name)
    elevations = []  # Liste pour stocker les élévations

    if location:
        latitude, longitude = location.latitude, location.longitude
        params = Topography.DEFAULT.copy()
        params["south"] = latitude - 0.2
        params["north"] = latitude + 0.2
        params["west"] = longitude - 0.2
        params["east"] = longitude + 0.2

        params["api_key"] = API_KEY_elevation
        city_topography = Topography(**params)
        city_topography.fetch()
        da = city_topography.load()  # Chargement des données topographiques

        # Récupération des élévations pour les points spécifiés
        for point in points:
            x_idx = int((point[1] - params["west"]) / (params["east"] - params["west"]) * (da.shape[2] - 1))
            y_idx = int((point[0] - params["south"]) / (params["north"] - params["south"]) * (da.shape[1] - 1))
            elevation = da[0, y_idx, x_idx].values
            elevations.append(elevation.item())  # Ajout de l'élévation à la liste

    else:
        print("Could not find coordinates for the city:", city_name)

    return elevations  # Retourne la liste des élévations

# Fonction pour lier les points, arretes et élévation dans un json
def merge_coordinates_elevations_to_json(points, elevations, edges, filename):
    folder = "cities"  # Nom du dossier pour enregistrer le fichier
    
    # Création d'un dictionnaire pour stocker les points et les arêtes
    data = {
        "points": [],
        "edges": []
    }
    
    # Ajouter les points avec leurs élévations
    for i in range(len(points)):
        if i < len(elevations):
            data["points"].append({"x": points[i][0], "y": points[i][1], "elevation": elevations[i]})
        else:
            data["points"].append({"x": points[i][0], "y": points[i][1], "elevation": None})
    
    # Ajouter les arêtes avec leurs longueurs
    for edge in edges:
        start_point = edge[0]
        end_point = edge[1]
        length = edge[2]
        is_oneway = edge[3]
        data["edges"].append({"start": {"x": start_point[0], "y": start_point[1]}, "end": {"x": end_point[0], "y": end_point[1]}, "length": length, "oneway": is_oneway})
    
    # Création du dossier s'il n'existe pas
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Enregistrement dans un fichier JSON
    file_path = os.path.join(folder, filename)
    with open(file_path, 'w') as f:
        json.dump(data, f)

    print(f"Data merged and saved to {file_path}")

# Fonction pour afficher le graph dans une carte folium
def visualize_graph_from_json(json_file, points_to_display):
    # Charger le fichier JSON
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Créer une carte Folium
    m = folium.Map()

    # Créer un ensemble pour stocker toutes les coordonnées des points à afficher
    all_points = set(points_to_display)

    # Ajouter les arêtes à la carte Folium et mettre à jour l'ensemble des coordonnées
    for edge in data["edges"]:
        start_point = (edge["start"]["x"], edge["start"]["y"])
        end_point = (edge["end"]["x"], edge["end"]["y"])
        all_points.add(start_point)
        all_points.add(end_point)
        if edge["oneway"]:
            # Si oneway est vrai, utiliser une couleur différente
            folium.PolyLine(locations=[start_point, end_point], color='red').add_to(m)
        else:
            folium.PolyLine(locations=[start_point, end_point], color='blue').add_to(m)


    # Ajouter les points spécifiés aux marqueurs avec les informations supplémentaires
    for point in points_to_display:
        elevation = None
        for pt in data["points"]:
            if pt["x"] == point[0] and pt["y"] == point[1]:
                elevation = pt["elevation"]
                break
        address = get_addresses_for_points([point])[0]
        popup_text = f"<b>Adresse:</b> {address}<br><b>Coordonnées:</b> {point}<br><b>Élévation:</b> {elevation}"
        folium.Marker(location=point, popup=popup_text).add_to(m)

    # Calculer les limites géographiques
    min_lat, min_lon = float('inf'), float('inf')
    max_lat, max_lon = float('-inf'), float('-inf')
    for lat, lon in all_points:
        min_lat = min(min_lat, lat)
        min_lon = min(min_lon, lon)
        max_lat = max(max_lat, lat)
        max_lon = max(max_lon, lon)

    # Zoomer sur les limites calculées
    m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])

    # Afficher la carte Folium
    display(m)

In [4]:
z = 5
ville = "Wazemmes"
pays = 'France'
lieu = ville + ',' + pays
G_ox = ox.graph_from_place(lieu, network_type="drive", simplify=False)
ville_node = list(G_ox.nodes)
ville_edge = list(G_ox.edges)

points = [(G_ox.nodes[node]['y'], G_ox.nodes[node]['x']) for node in ville_node]
reverse_points = [(G_ox.nodes[node]['x'], G_ox.nodes[node]['y']) for node in ville_node]
selected_node = random.sample(points, z)
reverse_selected_node = random.sample(reverse_points, z)

edge_list = [((G_ox.nodes[edge[0]]['y'], G_ox.nodes[edge[0]]['x']),
              (G_ox.nodes[edge[1]]['y'], G_ox.nodes[edge[1]]['x']),
              G_ox.edges[edge]['length'],
              G_ox.edges[edge].get('oneway', False))
             for edge in ville_edge]

In [5]:
elevations = get_city_topography(lieu, points)

In [6]:
# Appel de la fonction merge avec edge_list
merge_coordinates_elevations_to_json(points, elevations, edge_list, ville + ".json")

Data merged and saved to cities\Wazemmes.json


In [7]:
# Appel de la fonction pour visualiser le graphe avec les points spécifiés
visualize_graph_from_json(f"cities/{ville}.json", selected_node)

In [ ]:
import json

# Chemin vers le fichier JSON contenant le dictionnaire de points
chemin_fichier_json = f"cities/{ville}.json"

# Charger le dictionnaire depuis le fichier JSON
with open(chemin_fichier_json, "r") as fichier_json:
    dict_points = json.load(fichier_json)

# Conversion du dictionnaire en ensemble de tuples pour une recherche plus efficace
ensemble_points = {(point['x'], point['y']) for point in dict_points["points"]}

# Vérification de chaque point dans la liste
for point in selected_node:
    if point in ensemble_points:
        print(f"Le point {point} est présent dans le dictionnaire.")
    else:
        print(f"Le point {point} n'est pas présent dans le dictionnaire.")

